This script is used to plot the domains specified in the namelist.wps file.  plotgrids.ncl provides this feature, but sometimes I want to be able to visualize domains without having access to X11 forwarding on my linux system, or if I'm working on a windows machine and want to use python instead.

In [5]:
# import needed modules
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import re

# specify directories
my_wps_path = '/glade/u/home/zcleveland/scratch/NAM_soil-moisture/wpsv4.6.0/'  # path to my compiled wps directory
my_wrf_path = '/glade/u/home/zcleveland/scratch/NAM_soil-moisture/wrfv4.6.0/'  # path to my compiled wrf directory

In [20]:
def parse_namelist_wps(namelist_path):
    params = {}

    with open(namelist_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # remove comments and whitespace
            line = line.split('!')[0].strip()
            # check for '=', indicating parameter specification
            if '=' in line:
                param_line = line.split('=')
                param = param_line[0].strip()
                values = param_line[1].split(',')
                values = [val.strip() for val in values if val]
                params[param] = values
    return params


def calculate_domain_params(params):
    dx = params.get('dx', 0)
    dy = params.get('dy', 0)
    e_we = params.get('e_we', [0])
    e_sn = params.get('e_sn', [0])
    ref_lat = params.get('ref_lat', 0.0)
    ref_lon = params.get('ref_lon', 0.0)
    
    return dx, dy, ref_lat, ref_lon, e_we, e_sn

def plot_wrf_domains(params):
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.stock_img()
    ax.coastlines()

    for i in range(int(params['max_dom'][0])):
        lon_min = float(params['ref_lon'][0]) - (float(params['dx'][0]) * (float(params['we'][i]) - 1)) / 2
        lon_max = float(params['ref_lon'][0]) + (float(params['dx'][0]) * (float(params['we'][i]) - 1)) / 2
        lat_min = float(params['ref_lat'][0]) - (float(params['dy'][0]) * (float(params['sn'][i]) - 1)) / 2
        lat_max = float(params['ref_lat'][0]) + (float(params['dy'][0]) * (float(params['sn'][i]) - 1)) / 2

        ax.plot([lon_min, lon_max], [lat_min, lat_min], 'r')
        ax.plot([lon_min, lon_max], [lat_max, lat_max], 'r')
        ax.plot([lon_min, lon_min], [lat_min, lat_max], 'r')
        ax.plot([lon_max, lon_max], [lat_min, lat_max], 'r')

    # Add a white dot over the DC location (example)
    # dc_lon, dc_lat = -77.26, 38.56
    # ax.plot(dc_lon, dc_lat, 'wo')

    plt.show()

In [21]:
if __name__ == "__main__":
    namelist_path = os.path.join(my_wps_path, 'namelist.wps')  # Adjust the path as necessary
    params = parse_namelist_wps(namelist_path)
    dx, dy, ref_lat, ref_lon, e_we, e_sn = calculate_domain_params(params)
    plot_wrf_domains(dx, dy, ref_lat, ref_lon, e_we, e_sn)

TypeError: '<' not supported between instances of 'list' and 'float'